# multiply-ui

A GUI mock for the [Multiply](https://github.com/multiply-org) project based on
[Jupyter Widgets](https://ipywidgets.readthedocs.io) and [Bokeh](https://bokeh.pydata.org).

**Note**, you'll have to start the server first:
        
```
$ source activate multiply-ui
$ mui-server
```


Import the Multiply user interface components, called *mui* from now on

In [1]:
from multiply_ui import ui as mui

In [2]:
import json
mui.get_proc_parameters()

{'inputTypes': [{'id': 'S2_L1C',
   'name': 'Sentinel-2 MSI L1C',
   'timeRange': ['06-23-2015', None]}],
 'variables': [{'id': 'lai',
   'name': 'Effective Leaf Area Index',
   'unit': 'm2/m2',
   'description': 'Effective Leaf Area Index',
   'valueRange': '',
   'applications': []},
  {'id': 'cab',
   'name': 'Leaf Chlorophyll Content',
   'unit': 'ug/cm2',
   'description': 'Leaf Chlorophyll Content',
   'valueRange': '',
   'applications': []},
  {'id': 'cb',
   'name': 'Leaf Senescent Material',
   'unit': '',
   'description': 'Leaf Senescent Material',
   'valueRange': '',
   'applications': []},
  {'id': 'car',
   'name': 'Leaf Carotonoid Content',
   'unit': 'ug/cm2',
   'description': 'Leaf Carotonoid Content',
   'valueRange': '',
   'applications': []},
  {'id': 'cw',
   'name': 'Leaf Water Content',
   'unit': 'cm',
   'description': 'Leaf Water Content',
   'valueRange': '',
   'applications': []},
  {'id': 'cdm',
   'name': 'Leaf Dry Mass',
   'unit': 'g/cm2',
   'descr

A data UI may be used to search for data and to collect *file sets* to be processed.

In [3]:
mui.data_ui()

Box(children=(Box(children=(Label(value='Output variables'),), layout=Layout(display='flex', flex_flow='row', …

In [13]:
mui.get_proc_requests()

[]

Bring up the job execution UI that is used to submit processing jobs. In the example below, `duration` is just an example for a processing parameter. Instead, we would pass workflow-specific parameters here plus the list of input files to be processed.

In [14]:
mui.exec_ui()

interactive(children=(IntSlider(value=60, description='duration', max=1000, min=10, step=10), Button(descripti…

Now show all submitted jobs:

In [15]:
mui.Job.get_all()

Job ID,Duration,Progress,Status
0,60,,running
1,60,,running
2,60,,running
3,60,,running


Pick a certain job and interact with it:

In [16]:
job = mui.Job(2)
job

In [17]:
job.status

Job ID,Duration,Progress,Status
2,60,,running


In [18]:
job.cancel()

Job ID,Duration,Progress,Status
2,60,,cancelled


In [19]:
mui.Job(2).status

Job ID,Duration,Progress,Status
2,60,,cancelled


Let's pick another job and look at it's results (if processing has finished):

In [20]:
job = mui.Job(0)
job_results = job.results
job_results

Server error: HTTP Error 404: No results provided yet


We might also examine one of these in more detail

In [21]:
job_results.result('car')

AttributeError: 'NoneType' object has no attribute 'result'

# Updating Widgets

see https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Asynchronous.html#Updating-a-widget-in-the-background

In [22]:
import threading
from IPython.display import display
import ipywidgets as widgets
import time
progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)

def work(progress):
    total = 20
    mui.Job.execute_job(20)
    job = mui.Job(mui.Job.num_jobs() - 1)
    while progress.value < 1.0:
        time.sleep(0.4)
        progress.value = job.progress

thread = threading.Thread(target=work, args=(progress,))
display(progress)
thread.start()

FloatProgress(value=0.0, max=1.0)

This can be extended into an external component that monitors all processes:

In [23]:
mui.job_monitor()

# Custom Widgets

See https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Custom.html

In [ ]:
import ipywidgets as widgets
from traitlets import Unicode, validate

class HelloWidget(widgets.DOMWidget):
    _view_name = Unicode('HelloView').tag(sync=True)
    _view_module = Unicode('hello').tag(sync=True)
    _view_module_version = Unicode('0.1.0').tag(sync=True)

In [ ]:
%%javascript
require.undef('hello');

define('hello', ["@jupyter-widgets/base"], function(widgets) {

    var HelloView = widgets.DOMWidgetView.extend({

        // Render the view.
        render: function() {
            this.el.textContent = 'Hello World!';
        },
    });

    return {
        HelloView: HelloView
    };
});